## Create SVG

In [87]:
import pandas as pd
from pathlib import Path
import svgwrite
from svgwrite.container import Group
from svgwrite.shapes import Rect, Circle
import math
import functools

In [9]:
df = pd.read_excel(Path('../data/sitzung_aufteilung.xlsx'))

## Tests

In [11]:
# Check Ratio sum = 1
df_test = df.groupby('page').agg({'ratio': 'sum'})
df_test[df_test.ratio != 1]

,ratio
page,


In [12]:
df['gender'].unique()

array([0, 'p', 'm', 'w'], dtype=object)

## Draw

In [66]:
colors = {
    'm': '#24B39C',
    'w': '#6C43C0',
    'p': '#7dd1c3',
    0: '#cdcdd1'
}

In [97]:
# Precalculate Rect Position
for i in range(df['page'].min(), df['page'].max() + 1):
    df_sub = df[df.page == i]
    previousRatio = 0
    for j, row in df_sub.iterrows():
        df.loc[j, 'pos'] = previousRatio
        previousRatio += row['ratio']

In [128]:
size_desktop = {
    'cols': 14,
    'rows': 5,
    'name': 'd'
}

size_mobile = {
    'cols': 7,
    'rows': 10,
    'name': 'm'
}

In [129]:
platform = size_mobile

cols = platform['cols']
rows = platform['rows']
width = 70 
height = 100
padding = 10

svg = svgwrite.Drawing(
    Path('../export/csv_for_infographics/sitzung_130_%s.svg' % platform['name']),
    profile='tiny',
    size = (cols * (width + padding) + 1, rows * (height + padding) + 1)
)

for page in range(0, df['page'].max()):
    row = math.floor(page / cols)
    col = page % cols

    g = svg.add(Group(
        id='%s-%s' % (row, col),
        transform="translate(%s, %s)" % (col * (width + padding) + 1, row * (height + padding) + 1)
        # transform="translate(%s, %s)" % (col * (width + padding), row * (height + padding))
    ))
    
    g.add(Rect(
        insert=(0, 0),
        size=(width, height),
        stroke='#000000',
        stroke_width = 1,
        fill='none'
    ))

    for i, df_row in df[df.page == (page + 1)].iterrows():
        g.add(Rect(
            insert=(0, height * df_row['pos']),
            size=(width, height * df_row['ratio']),
            stroke_width = 0,
            fill=colors[df_row['gender']]
            ))    

svg.save()

## Bubble Chart

In [101]:
clusters = [{
        'point_count': 3047,
        'fill': colors['w']
    },
    {
        'point_count': 8476,
        'fill': colors['m']
    }
]

# clusters = [{
#         'point_count': 17422,
#         'fill': colors['w']
#     },
#     {
#         'point_count': 43343,
#         'fill': colors['m']
#     }
# ]

In [102]:
point_radius = 10

svg = svgwrite.Drawing(
    Path('../export/csv_for_infographics/cloud.svg'),
    profile='tiny',
)

def draw_cluster(cluster):
    r = 0
    counter = 0
    group = 0

    clusterGroup = Group()

    while counter < cluster['point_count']:
        r = point_radius * (group + 0) * 2
        if group == 0:
            # Center Circle
            count = 1
        else:
            U = 2 * r * (group + 0) * math.pi
            count = math.floor(math.floor(U) / r)

        g = clusterGroup.add(Group(id = group))

        # Now loop count
        for j in range(0, count):
            alpha = 360 / count * j
            x = r * math.sin(math.pi * 2 * alpha / 360)
            y = r * math.cos(math.pi * 2 * alpha / 360)

            g.add(Circle((x, y), point_radius,
            fill=cluster['fill'],
            stroke='none',
            stroke_width = 0,
            ))

            counter += 1
            if counter >= cluster['point_count']:
                break
        
        group += 1

    viewbox = group * point_radius * 2
    return clusterGroup, viewbox

# Create Clusters
groups = []
for cluster in clusters:
    g, viewbox = draw_cluster(cluster)
    groups.append({'g': g, 'viewbox': viewbox})

# Add to SVG
for i, c in enumerate(groups):
    
    c['g'].translate(c['viewbox'] * (-1 if i % 2 == 0 else 1), 0)
    svg.add(c['g'])

# Set ViewBox
width = functools.reduce(lambda x, y: {'viewbox': x['viewbox'] + y['viewbox']}, groups)['viewbox']
svg.viewbox(groups[0]['viewbox'] * -2, groups[0]['viewbox'] * -2, width * 2, width * 2)

svg.save()

## Grid

In [110]:
# Load Sitzungen
df_votum_raw = pd.concat([
    pd.read_csv(Path('../export/votum/votum_0.csv')),
    pd.read_csv(Path('../export/votum/votum_1.csv'))
])

# Remove non members (mostly former members who are now in the Regierungsrat)
df_votum_raw = df_votum_raw[df_votum_raw.ismember == True]

# Typecast
df_votum_raw['sitzung_date'] = pd.to_datetime(df_votum_raw['sitzung_date'])

# Remove empty texts
df_votum_raw = df_votum_raw[df_votum_raw.text.notna()]

# Replace CVP with Die Mitte
df_votum_raw.loc[df_votum_raw.partei.str.lower() == 'cvp', 'partei'] = "Die Mitte"

df_votum_raw['g'] = df_votum_raw['geschlecht']
df_votum_raw.loc[df_votum_raw.funktion.notna(), 'g'] = 'p'
df_votum_raw.loc[df_votum_raw.ismember == False, 'g'] = 'nomember'

len(df_votum_raw)

76821

In [134]:
df = df_votum_raw[df_votum_raw.sitzung_date >= '2019-05-01'].reset_index()

cols = 100
size = {
    'width': 20,
    'height': 30,
    'padding': 1
}

svg = svgwrite.Drawing(
    Path('../export/csv_for_infographics/grid.svg'),
    profile='tiny',
    # viewBox=(10, 10, 10, 10)
    size = (
        cols * (size['width'] + size['padding']),
        math.ceil(len(df) / cols) * (size['height'] + size['padding'])
    )
)

for i, row in df.iterrows():
    gridrow = math.floor(i / cols)
    gridcol = i % cols

    svg.add(Rect(
        insert=(
            gridcol * (size['width'] + size['padding']),
            gridrow * (size['height'] + size['padding']),
        ),
        size=(size['width'], size['height']),
        stroke='#000',
        stroke_width = 1,
        fill=colors[row['g']]
    ))

svg.save()